# 文本预处理

## 读入文本

· 正则表达式：[学习资源](https://www.runoob.com/regexp/regexp-syntax.html)

· python文件读写：
```python
with open(file_location) as f : 
```

普通的open（不用with）的打开文件操作，如果读写过程中出现异常，则后面的close则不会被调用，从而占用计算机资源。

python中的with，则封装了close方法，读写完毕或者失败之后都会close。


## 分词

`string.split()`的用法

列表表达式（list comprehension):`[sentence.split(' ') for sentence in sentences]`

## 建立字典

构建字典：将每个词映射到唯一的索引编号。

`collections.Counter`：[doc链接](https://docs.python.org/zh-cn/3/library/collections.html#collections.Counter)

>一个计数工具，提供快速和方便的计数。比如：
```python
cnt = Counter()
for word in ['red', 'blue', 'red', 'green', 'blue', 'blue']:
    cnt[word] += 1
cnt
```
Counter({'blue': 3, 'red': 2, 'green': 1})

一个 Counter 是一个 dict 的子类，用于计数可哈希对象。它是一个集合，元素像字典键(key)一样存储，它们的计数存储为值。计数可以是任何整数值，包括0和负数。

## 词转为索引

利用字典，我们可以将源文本中的句子从单词序列转换为索引序列。

# 语言模型

给定一段长度为$T$的文本中的词依次为$w_{1}, w_{2}, \dots, w_{T}$，语言模型将计算该序列的概率：

$P\left(w_{1}, w_{2}, \dots, w_{T}\right)$

## 语言模型的计算

$P\left(w_{1}, w_{2}, \ldots, w_{T}\right)=\prod_{t=1}^{T} P\left(w_{t} | w_{1}, \ldots, w_{t-1}\right)$

为了计算语言模型，我们需要计算词的概率，及一个词在给定前几个词下的条件概率，即语言模型的参数。

## n元语法

马尔科夫假设：一个词的出现只与其前n个词有关（即n阶马尔科夫链）。

n太大，计算复杂度高；n太小，模型过于简单，可能不符合实际。


## 时序数据的采样

### 随机采样

在训练中我们需要每次随机读取小批量的样本和标签。与之前章节的实验数据不用的是，时序数据的一个batch（batch_size个）通常包含连续的字符（num_step个）。

随机采样：每个样本时原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定毗邻。**因此我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要初始化隐藏状态。**

```python
def data_iter_random(data,batch_size,num_steps):
    num_sample = (len(data) - 1) // num_steps #减1是因为最后一个字符不能包含在样本中。
    example_indices = list(range(num_sample))
    random.shuffle(batch_head_index)
    
    def _data(i): #单下划线开头的method意为：该method仅供内部使用
        return data[i:i+batch_size]
    
    epoch_size = num_sample // batch_size
    for i in range(epoch_size):
        i = i * batch_size
        batch_indices = example_indices [i:i+batch_size]
        X = [_data(j*num_steps) for j in batch_indices]
        Y = [_data(j*num_steps+1) for j in batch_indices]
        yield torch.tensor(X),torch.tensor(Y)
```

The `shuffle()` method takes a sequence (list, string, or tuple) and reorganize the order of the items. Note: This method changes the original list/tuple/string, it does not return a new list/tuple/string.

### 相邻采样

相邻的两个随机小批量在原始序列上的位置相毗邻。这时候，可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。

这对实现循环神经网络造成了两方面影响：一方面，在训练模型时，只需要在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。


```python
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
```


# 循环神经网络基础

n元语法中，增大n会导致模型参数的数量呈指数增长。循环神经网络通过隐藏状态存储之前时间步的信息。

现在，我们考虑数据存在时间相关性的情况。假设$\boldsymbol{X}_{t} \in \mathbb{R}^{n \times d}$是序列中时间步$t$的小批量输入，$\boldsymbol{H}_{t} \in \mathbb{R}^{n \times h}$是该时间步的隐藏变量。与多层感知机不同的是，我们保存上一时间步的隐藏变量$\boldsymbol{H}_{t-1}$。时间步$t$的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定。

$\boldsymbol{H}_{t}=\phi\left(\boldsymbol{X}_{t} \boldsymbol{W}_{x h}+\boldsymbol{H}_{t-1} \boldsymbol{W}_{h h}+\boldsymbol{b}_{h}\right)$

在时间步$t$，输出层的输出和多层感知机中的计算类似：
$\boldsymbol{O}_{t}=\boldsymbol{H}_{t} \boldsymbol{W}_{h q}+\boldsymbol{b}_{q}$。

循环神经网络的参数包括隐藏层的权重$\boldsymbol{W}_{x h} \in \mathbb{R}^{d \times h}$,$\boldsymbol{W}_{h h} \in \mathbb{R}^{h \times h}$和偏差$\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h}$，以及输出层的权重$\boldsymbol{W}_{h q} \in \mathbb{R}^{h \times q}$和偏差$b_{q} \in \mathbb{R}^{1 \times q}$。

值得一提的是，即便在不同时间步，循环神经网络也始终使用这些模型参数。因此，循环神经网络模型参数的数量不随时间步的增加而增长。

![含隐藏状态的循环神经网络](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter06/6.2_rnn.svg)

基于字符级循环神经网络的语言模型:

![基于字符级循环神经网络的语言模型](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter06/6.2_rnn-train.svg)

小批量中样本数为1，文本序列为：。对每个时间步的输出层进行softmax运算，然后使用交叉熵损失函数计算loss。